In [13]:
import numpy as np
import nltk

nltk.download('punkt')
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
stemmer = PorterStemmer()

def tokenize(sentence):

    return nltk.word_tokenize(sentence)


def stem(word):

    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):

    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag

In [15]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out



In [16]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



with open('intents.json', 'r') as f:
  intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

#print(len(xy), "patterns")
#print(len(tags), "tags:", tags)
#print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


#print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

#print(f'training complete. file saved to {FILE}')

71 8
Epoch [100/1000], Loss: 1.6990
Epoch [200/1000], Loss: 0.2071
Epoch [300/1000], Loss: 0.0054
Epoch [400/1000], Loss: 0.0060
Epoch [500/1000], Loss: 0.0018
Epoch [600/1000], Loss: 0.0059
Epoch [700/1000], Loss: 0.0004
Epoch [800/1000], Loss: 0.0011
Epoch [900/1000], Loss: 0.0003
Epoch [1000/1000], Loss: 0.0004


In [17]:
print(f'final loss: {loss.item():.4f}')

final loss: 0.0004


In [18]:
print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [19]:
import random
import json

import torch



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Bot"
print("Let's chat! (type 'quit' to exit)")
while True:
    
    sentence = input("You: ")
    if sentence == "quit":
        print(f"{bot_name}: Thank you! Have a nice stay.")
        break
    
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == "Booking":
              
              print(f"{bot_name}: Enter your name")
              user_name=input("You: ")
              print(f"{bot_name}: Enter your mobile number")
              user_number=input("You: ")
              if len(user_number)!=10 :
                print(f"{bot_name}: Enter Valid number")
                user_number=input("You: ")
              print(f"{bot_name}: Enter your Mail address")
              user_email=input("You: ")
              import random
              n = random.randint(100000,900000)
              print(f"{bot_name}: Booking confirmed with booking ID: {n}, details sent to {user_number} and {user_email}")
              

              break

            elif tag == intent["tag"]:
              print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: HI
Bot: Hi there, how can I help?
You: I want to know about the facilities in the hotel
Bot: I do not understand...
You: What are the facilities availble here
Bot: We have free WiFi , hot water , Complementary breakfast  and a Travel advisory desk here
You: what about breakfast
Bot: We have complementary breakfast which will be served from morning 8 A.M.
You: How can i book a room
Bot: We accept most major credit cards,debit cards, cash ,and Paypal
You: please book a room for me
Bot: Enter your name
You: Naveen 
Bot: Enter your mobile number
You: 70956768908
Bot: Enter Valid number
You: 7095676898
Bot: Enter your Mail address
You: naveen.sanapala@gmail.com
Bot: Booking confirmed with booking ID: 505375, details sent to 7095676898 and naveen.sanapala@gmail.com
You: Thank you
Bot: Happy to help!
You: quit
Bot: Thank you! Have a nice stay.
